Reference: Github of Isabel Segura Bedmar https://github.com/isegura/BasicNLP

Libraries: 
Spacy https://spacy.io
Spacy-lookup  https://github.com/mpuig/spacy-lookup"""

Source: https://support.prodi.gy/t/fuzzy-partial-matching-with-phrasematcher-ner-task/1084

Source: https://towardsdatascience.com/natural-language-processing-for-fuzzy-string-matching-with-python-6632b7824c49

In [2]:
from google.colab import drive
drive.mount("/content/drive/")
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
drive  sample_data


In [3]:
!pip install spacy-lookup    #For Dictionary based NER ,  Successfully installed flashtext-2.7 spacy-lookup-0.1.0

  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9299 sha256=fca4f9833a82bc80d4f5a504ff0d9705c29acc35854fec100f257807e3f9823e
  Stored in directory: /root/.cache/pip/wheels/37/db/d7/fe74f7cb8e5c3afed90fe6f4967c933a6f13d81ab6b3d3128c
Successfully built flashtext


In [6]:
"""Reference: Github of Isabel Segura Bedmar https://github.com/isegura/BasicNLP
Libraries: 
Spacy https://spacy.io
Spacy-lookup  https://github.com/mpuig/spacy-lookup"""

import pandas as pd
from tabulate import tabulate
import re
import csv
import collections

!pip3 install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import spacy
from spacy_lookup import Entity
!pip install -U spacy #spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)   1 time
!pip install spacy
!python -m spacy download es          #Every time it reload    ,   Successfully installed es-core-news-sm-2.2.5
!pip install spacy-lookup    #For Dictionary based NER ,  Successfully installed flashtext-2.7 spacy-lookup-0.1.0

nlp = spacy.load('es')          #"en_core_web_sm"
print('spacy.es loaded')


Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.3.2)
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')


FileNotFoundError: ignored

In the notebook **"0904_terminado_Crear tsv a partir de text files.ipynb"**, there is a section called **"Create a tsv from all text files (train and dev)"**. It is necessary to execute it before load the independent text files.

https://colab.research.google.com/drive/1JhAqsmZBLJzpb0lA8BlU2Kzldv6WQ0b5#scrollTo=DLKJlTsxQ4YB

1. Articles, Diagnostics/Procedures, train -- train_dev/train/text_files/... --> train_dev/train/train_articles_codiesp.csv
2. Articles, Diagnostics/Procedures, dev -- train_dev/dev/text_files/... --> train_dev/dev/dev_articles_codiesp.csv


#Load data

In [5]:
def load_articles(path):
  df = pd.read_csv(path, sep=",", engine="python", encoding='utf-8')
  #print("--- Loaded dataset:", path)
  #print("--- Number of rows is " + str(df.shape[0]) + " x number of columns " + str(df.shape[1]))
  #print(tabulate(df.head(), headers='keys', tablefmt='psql'))
  return df

def load_data(path, names):
  df = pd.read_csv(path, sep="\t", names=names)
  #print("--- Loaded dataset:", path)
  #print("--- Number of rows is " + str(df.shape[0]) + " x number of columns " + str(df.shape[1]))
  #print(tabulate(df.head(), headers='keys', tablefmt='psql'))
  return df




"""Paths"""
localpath = '/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp_code/SubtrackD'
#path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'
train_file_articles = localpath + '/final_dataset_v3_to_publish/train/train_articles_codiesp.csv'
dev_file_articles = localpath + '/final_dataset_v3_to_publish/dev/dev_articles_codiesp.csv'
test_file_articles = localpath + '/final_dataset_v3_to_publish/test/test_articles_codiesp.csv'

names_codeD=["Code", "Es-description", "En-description"]   # Columns: code \t es-description \t en-description
names_codeP=["Code", "Es-description", "En-description", "Approach?"]   # Columns: code \t es-description \t en-description \t approach?
file_codeD = localpath + '/codiesp_codes/codiesp-D_codes.tsv'



"""- Load Articles train"""
df_train_articles = load_articles(train_file_articles)
#print(tabulate(df_train_articles.head(), headers='keys', tablefmt='psql'))
df_dev_articles = load_articles(dev_file_articles)
#print(tabulate(df_dev_articles.head(), headers='keys', tablefmt='psql'))
df_test_articles = load_articles(test_file_articles)
#print(tabulate(df_test_articles.head(), headers='keys', tablefmt='psql'))

"""--- Load Code, train, test"""
df_codeD = load_data(file_codeD, names_codeD)
#print(tabulate(df_codeD.head(), headers='keys', tablefmt='psql'))

FileNotFoundError: ignored

#Dictionary based NER with Spacy using entities from CIE-10 dictionary

In [ ]:
def ner(text):
  text = text.lower() #en el caso de poner todo minusculas
  document = nlp(text)

  result = []
  for entity in document.ents:                        #divide document into entities
      #print('Type: {}, Value: {}, star: {}, end: {}'.format(entity.label_, entity.text,entity.start_char, entity.end_char))   #start_char and end_char are the position of the first and last character of an entity, respect the document begining.
      result.append(entity.text)   #start_char and end_char are the position of the first and last character of an entity, respect the document begining.
      #print(result)

  return(result)


def load_dictionary(df, language_index):
  """This function reads a df and save the fourth column into a list. 
  This list is an input parameter, which is modified. We need to pass
  the list as parameter, because if the list is returned, 
  its type becomes to NoneType"""
  dictionary=[]
  for index, row in df.iterrows():                                  
    #print(row[1])
    dictionary.append(row[language_index].lower())          #language_index : 1 spanish and 2 english
  #print(dictionary)
  #print('length of the dictionary loaded: ', len(dictionary))
  return(dictionary)



def Dict_code_reference(df):    #Make a dictionary {'A00.0': 'Cólera debido a Vibrio cholerae 01, biotipo cholerae ', 'A00.1': 'Cólera debido a Vibrio cholerae 01, biotipo El Tor',...
  code_list = []
  reference_list = []

  for index, row in df.iterrows():
    code_list.append(row[0])
    reference_list.append(row[1])    #1 spanish, 2 english
  #print(code_list, reference_list)
  
  #Converting a list to dictionary with list elements as values in dictionary and keys are enumerated index starting from 0 i.e. index position of element in list
  dict_code_reference = { code_list[i] : reference_list[i] for i in range(0, len(code_list)) }
  #print(dict_code_reference)

  return(dict_code_reference)



##Spanish

print("---Load dictionary spanish")
diagnostics_dictionary = load_dictionary(df_codeD, language_index=1)

#Add Named Entities metadata to Doc objects in Spacy. First, we load the model and replace the NER module with de entity diagnosticEnt. We do this to avoid overlapping of entities. Then, we also add procedureEnt.
diagnosticEnt = Entity(keywords_list=diagnostics_dictionary,label="DIAGNOSTICO")  #Detect Named Entities using dictionaries. We can process a text and show its entities.


nlp.replace_pipe("ner", diagnosticEnt)   #We replace the common entities with diagnostics
print('entities loaded in nlp')
"""
##English

print("---Load dictionary english")
diagnostics_dictionary = load_dictionary(df_codeD, language_index=2)

#Add Named Entities metadata to Doc objects in Spacy. First, we load the model and replace the NER module with de entity diagnosticEnt. We do this to avoid overlapping of entities. Then, we also add procedureEnt.
diagnosticEnt = Entity(keywords_list=diagnostics_dictionary,label="DIAGNOSTICO")  #Detect Named Entities using dictionaries. We can process a text and show its entities.


nlp.replace_pipe("ner", diagnosticEnt)   #We replace the common entities with diagnostics
print('entities loaded in nlp')
"""



---Load dictionary spanish
entities loaded in nlp


'\n##English\n\nprint("---Load dictionary english")\ndiagnostics_dictionary = load_dictionary(df_codeD, language_index=2)\n\n#Add Named Entities metadata to Doc objects in Spacy. First, we load the model and replace the NER module with de entity diagnosticEnt. We do this to avoid overlapping of entities. Then, we also add procedureEnt.\ndiagnosticEnt = Entity(keywords_list=diagnostics_dictionary,label="DIAGNOSTICO")  #Detect Named Entities using dictionaries. We can process a text and show its entities.\n\n\nnlp.replace_pipe("ner", diagnosticEnt)   #We replace the common entities with diagnostics\nprint(\'entities loaded in nlp\')\n'

#NER into Reference and code

In [ ]:
print(tabulate(df_codeD.head(), headers='keys', tablefmt='psql'))    #Dataframe with code and description

+----+--------+------------------------------------------------------+----------------------------------------------------+
|    | Code   | Es-description                                       | En-description                                     |
|----+--------+------------------------------------------------------+----------------------------------------------------|
|  0 | A00.0  | Cólera debido a Vibrio cholerae 01, biotipo cholerae | Cholera due to Vibrio cholerae 01, biovar cholerae |
|  1 | A00.1  | Cólera debido a Vibrio cholerae 01, biotipo El Tor   | Cholera due to Vibrio cholerae 01, biovar eltor    |
|  2 | A00.9  | Cólera, no especificado                              | Cholera, unspecified                               |
|  3 | A01.00 | Fiebre tifoidea, no especificada                     | Typhoid fever, unspecified                         |
|  4 | A01.01 | Meningitis tifoidea                                  | Typhoid meningitis                                 |
+----+--

In [ ]:
dict_code_reference = Dict_code_reference(df_codeD)

In [ ]:
dict_code_reference

{'A00.0': 'Cólera debido a Vibrio cholerae 01, biotipo cholerae ',
 'A00.1': 'Cólera debido a Vibrio cholerae 01, biotipo El Tor',
 'A00.9': 'Cólera, no especificado',
 'A01.00': 'Fiebre tifoidea, no especificada',
 'A01.01': 'Meningitis tifoidea',
 'A01.02': 'Fiebre tifoidea con afectación cardiaca',
 'A01.03': 'Neumonía tifoidea',
 'A01.04': 'Artritis tifoidea',
 'A01.05': 'Osteomielitis tifoidea',
 'A01.09': 'Fiebre tifoidea con otras complicaciones',
 'A01.1': 'Fiebre paratifoidea A',
 'A01.2': 'Fiebre paratifoidea B',
 'A01.3': 'Fiebre paratifoidea C',
 'A01.4': 'Fiebre paratifoidea, no especificada',
 'A02.0': 'Enteritis debida a Salmonella',
 'A02.1': 'Sepsis debida a Salmonella',
 'A02.20': 'Infecciones localizadas debidas a Salmonella, no especificadas',
 'A02.21': 'Meningitis por Salmonella',
 'A02.22': 'Neumonía por Salmonella',
 'A02.23': 'Artritis por Salmonella',
 'A02.24': 'Osteomielitis por Salmonella',
 'A02.25': 'Pielonefritis por Salmonella',
 'A02.29': 'Salmonella c

In [ ]:
def Find_code_preditions(df, dictionary, path):
  with open(path, 'wt') as out_file:
      tsv_writer = csv.writer(out_file, delimiter='\t')
      for index, row in df.iterrows():
          abs_id = row[0]
          text = row[1]
          result = ner(text)
          length = len(result)
          if (length >0):                                 #no empty text_files
            for i in range(length):
              entity = result[i]
              min_score = 50                              #score ranges
              max_score = -1
              for code, reference in dict_code_reference.items():    
                score = fuzz.ratio(entity, reference)
                if (score > min_score)&(score> max_score):
                  max_reference = reference
                  max_score = score
                  max_code = code
              
              #print(abs_id, max_code, max_score, max_reference, entity)
              #print("----", abs_id, score, result[i]) 
              tsv_writer.writerow([abs_id, max_code, max_score, max_reference, entity])





#Find_code_preditions(df_train_articles, dict_code_reference, localpath +'/predictions/trainD_prediction_codes.tsv')
#Find_code_preditions(df_dev_articles, dict_code_reference, localpath +'/predictions/devD_prediction_codes.tsv')
Find_code_preditions(df_test_articles, dict_code_reference, localpath +'/predictions/testD_prediction_codes.tsv')



#Load predictions

In [ ]:
df_train_prediction = pd.read_csv(localpath +'/predictions/trainD_prediction.tsv', sep="\t", header=None)
df_dev_prediction = pd.read_csv(localpath +'/predictions/devD_prediction.tsv', sep="\t", header=None)
df_test_prediction = pd.read_csv(localpath +'/predictions/testD_prediction.tsv', sep="\t", header=None)

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher
nlp_blank = spacy.blank('en')

drug_list = diagnostics_dictionary
matcher = PhraseMatcher(nlp_blank.vocab)
matcher.add('DRUG', None, *[nlp_blank(entity_i) for entity_i in drug_list])


doc = nlp_blank("dolores de cabeza con brucella")
matches = matcher(doc)

for m_id, start, end in matches:
    entity = doc[start : end] 
    print((entity.text, entity.start_char, entity.end_char, nlp_blank.vocab.strings[m_id]))


In [ ]:
print(matches)

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


text = "A patient was prescribed Adepend 5mg, Alfuzosin 20ml and co-magaldrox 5 mg"

for query in drug_list:
    print(process.extractOne(query, text.split()))